In [3]:
import numpy as np
import pandas as pd
import time
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import gezi
from gezi import tqdm
from src import util
from src.config import *
gezi.init_flags()

No module named 'tensorflow.python.keras.layers.preprocessing'


In [4]:
np.random.seed(2022)

In [5]:
df = pd.read_feather('../input/feedback-prize-2021/train.fea')

In [6]:
df[df.id=='00066EA9880D'][['discourse_type_id', 'discourse_type', 'start']]

,discourse_type_id,discourse_type,start
82076,5,Lead,0
82077,3,Position,77
82078,1,Claim,106
82079,2,Evidence,124
82080,1,Claim,243
82081,2,Evidence,258
82082,1,Claim,410
82083,2,Evidence,438
82084,4,Concluding Statement,560


In [7]:
id_ = 'C196DA0C7688'

In [8]:
MAX_LEN = 2000
id = None
fes = []
fe = {}
pre_idx = 0
for i, row in tqdm(enumerate(df.itertuples()), total=len(df)):
  row = row._asdict()
  if row['id'] != id:
    if fe:
      for j in range(pre_idx, len(fe['text'].split())):
        fe['para_types'][j] = 0
      fe['para_types'] = fe['para_types'][:j + 1]
      fe['start'] = fe['start'][:j + 1]
      assert len(fe['para_types']) == len(fe['text'].split())
      assert min(fe['para_types']) >= 0
      fes.append(fe)
    fe = {}
    fe['id'] = row['id']
    fe['para_types'] = [-1] * MAX_LEN
    fe['start'] = [0] * MAX_LEN
    fe['text'] = row['text_']
    pre_idx = 0
    id = row['id']
  predictionstring = row['predictionstring']
  positions = predictionstring.split()
  start, end = int(positions[0]), int(positions[-1])
  fe['start'][start] = 1
  if start > pre_idx:
    for j in range(pre_idx, start):
      fe['para_types'][j] = 0
  for j in range(start, end + 1):
    fe['para_types'][j] =  row['discourse_type_id']
  pre_idx = end + 1
if fe:
  for j in range(pre_idx, len(fe['text'].split())):
    fe['para_types'][j] = 0
  fe['para_types'] = fe['para_types'][:j + 1]
  fe['start'] = fe['start'][:j + 1]
  assert len(fe['para_types']) == len(fe['text'].split())
  assert min(fe['para_types']) >= 0, fe['id']
  fes.append(fe)

  0%|          | 0/144293 [00:00<?, ?it/s]

In [9]:
d = pd.DataFrame(fes)

In [10]:
d[d.id==id_]['text'].values[0]

'The author has been studying Venus to find out if humans can live there and there are some reason that humans cant live there.\n\nThe author says that Venus has many feature like earth and that it may be possible for humans to live there. Even though they have seen what could happen like the bad part of living there, they are still trying to find a way to make humans live there. Venus is like Earths "twin" because it is the closest planet to the earth unlike the other planets. Also, Venus atmosphere of carbon dioxide is 97 percent.\n\nVenus may be worth studying for but despite the danger that it may have to any human could be bad and even though the author says that they are going to have to make humans a part of there studyies so they can find a way to live there. In the text it says that "numerous factors contribute to Venus\'s reputation as a challenging planet for humans to study, despite the proximity to us" saying that they will have to study humans no matter what.\n\nTherefore

In [11]:
d

,id,para_types,start,text
0,423A1CA112E2,"[0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Phones\n\nModern humans today are always on th...
1,A8445CABFECE,"[0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Phones & Driving\n\nDrivers should not be able...
2,6B4F7A0165B9,"[0, 0, 0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Cell Phone Operation While Driving\n\nThe abil...
3,E05C7F5C1156,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",People are debating whether if drivers should ...
4,50B3435E475B,"[0, 0, 0, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Texting and driving\n\nOver half of drivers in...
...,...,...,...,...
15589,8F4B595CF9E7,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Do you ever want more opinions and options whe...
15590,6B5809C83978,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...",Has anyone ever gave you advice? Was the advic...
15591,408A7D3D2EEC,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Imagine seeking advice from multiple people an...
15592,AFEC37C2D43F,"[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",There has been at least one point in everyone'...


In [12]:
d.reset_index().to_feather('../input/feedback-prize-2021/para_label.fea')

In [13]:
d['part'] = [np.random.randint(19) for _ in range(len(d))]

In [14]:
gezi.init_flags()

In [15]:
def find_span(words, start, end):
  start2, end2 = None, None
  idx = 0
  for i, word in enumerate(words):
    if word == FLAGS.br:
      continue
    else:
      if idx == start:
        start2 = i
      if idx == end:
        end2 = i
      idx += 1
  return start2, end2

fes = []
for row in tqdm(d.itertuples(), total=len(d)):
  row = row._asdict()
  para_id = 0
  pre_type = None
  words = util.get_words(row['text'])
  pre_idx =0
  for i, para_type in enumerate(row['para_types']):
    if (para_type != pre_type) or (row['start'][i] == 1):
      if pre_type != None:
        fe = {}
        fe['id'] = row['id']
        fe['part'] = row['part']
        fe['para_id'] = para_id
        fe['para_type'] = pre_type
        fe['start'] = pre_idx
        fe['end'] = i
        start2, end2 = find_span(words, pre_idx, i)
        fe['start2'] = start2
        fe['end2'] = end2
        fe['para'] = ' '.join(words[start2: end2])
        fes.append(fe)
        para_id += 1
        pre_idx = i
      pre_type = para_type
  i += 1
  fe = {}
  fe['id'] = row['id']
  fe['part'] = row['part']
  fe['para_id'] = para_id
  fe['para_type'] = pre_type
  fe['start'] = pre_idx
  fe['end'] = i
  start2, end2 = find_span(words, pre_idx, i)
  fe['start2'] = start2
  fe['end2'] = end2
  fe['para'] = ' '.join(words[start2: end2])
  fes.append(fe)
      

  0%|          | 0/15594 [00:00<?, ?it/s]

In [16]:
d = pd.DataFrame(fes)

In [17]:
d

,id,part,para_id,para_type,start,end,start2,end2,para
0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR]
1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone....
2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when stu...
3,423A1CA112E2,13,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...
4,423A1CA112E2,13,4,2,76,139,80,145.0,"When people have phones, they know about certa..."
...,...,...,...,...,...,...,...,...,...
175155,4C471936CD75,7,10,2,576,828,582,834.0,seeking multiple opinions before making a hard...
175156,4C471936CD75,7,11,3,828,839,834,845.0,it is better to seek multiple opinions instead...
175157,4C471936CD75,7,12,2,839,903,845,911.0,The impact of asking people to help you make a...
175158,4C471936CD75,7,13,0,903,905,911,913.0,"Of course,"


In [18]:
d[d.id==id_].para.values

array(['The author has been studying Venus to find out if humans can live there and there are some reason that humans cant live there. [BR] [BR]',
       'The author says that Venus has many feature like earth and that it may be possible for humans to live there. Even though they have seen what could happen like the bad part of living there, they are still trying to find a way to make humans live there. Venus is like Earths "twin" because it is the closest planet to the earth unlike the other planets. Also, Venus atmosphere of carbon dioxide is 97 percent. [BR] [BR] Venus may be worth studying for but despite the danger that it may have to any human could be bad and even though the author says that they are going to have to make humans a part of there studyies so they can find a way to live there. In the text it says that "numerous factors contribute to Venus\'s reputation as a challenging planet for humans to study, despite the proximity to us" saying that they will have to study huma

In [19]:
ids = []
texts = []
id = None
words = []
for row in tqdm(d.itertuples(), total=len(d)):
  row = row._asdict()
  if row['id'] != id:
    if words:
      ids.append(id)
      texts.append(' '.join(words))
    words = []
    id = row['id']
  words.extend(row['para'].split(' '))
if words:
  ids.append(id)
  texts.append(' '.join(words))
d_text = pd.DataFrame({
  'id': ids,
  'text': texts
})
d_text['text'] = d_text.text.apply(lambda x: x.replace('\n', FLAGS.br))

  0%|          | 0/175160 [00:00<?, ?it/s]

In [20]:
d_text[d_text.id==id_].values[0]

array(['C196DA0C7688',
       'The author has been studying Venus to find out if humans can live there and there are some reason that humans cant live there. [BR] [BR] The author says that Venus has many feature like earth and that it may be possible for humans to live there. Even though they have seen what could happen like the bad part of living there, they are still trying to find a way to make humans live there. Venus is like Earths "twin" because it is the closest planet to the earth unlike the other planets. Also, Venus atmosphere of carbon dioxide is 97 percent. [BR] [BR] Venus may be worth studying for but despite the danger that it may have to any human could be bad and even though the author says that they are going to have to make humans a part of there studyies so they can find a way to live there. In the text it says that "numerous factors contribute to Venus\'s reputation as a challenging planet for humans to study, despite the proximity to us" saying that they will have 

In [21]:
len(d_text[d_text.id==id_].text.values[0].split())

228

In [22]:
len(d_text[d_text.id==id_].text.values[0].replace(FLAGS.br, '\n').split())

222

In [23]:
d = pd.merge(d, d_text, on=['id'], how='left')
d['pid'] = range(len(d))
d['para'] = d.para.apply(lambda x: x.replace('\n', FLAGS.br))

In [24]:
d = pd.merge(d, df[['id', 'kfold', 'cluster']].drop_duplicates(), on='id', how='inner')

In [25]:
d['worker'] = d['part'] * 5 + d['kfold']

In [26]:
d

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR],Phones [BR] [BR] Modern humans today are alway...,0,2,8,67
1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone....,Phones [BR] [BR] Modern humans today are alway...,1,2,8,67
2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [BR] [BR] Modern humans today are alway...,2,2,8,67
3,423A1CA112E2,13,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [BR] [BR] Modern humans today are alway...,3,2,8,67
4,423A1CA112E2,13,4,2,76,139,80,145.0,"When people have phones, they know about certa...",Phones [BR] [BR] Modern humans today are alway...,4,2,8,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175155,4C471936CD75,7,10,2,576,828,582,834.0,seeking multiple opinions before making a hard...,"In ancient times, and also still today in some...",175155,4,1,39
175156,4C471936CD75,7,11,3,828,839,834,845.0,it is better to seek multiple opinions instead...,"In ancient times, and also still today in some...",175156,4,1,39
175157,4C471936CD75,7,12,2,839,903,845,911.0,The impact of asking people to help you make a...,"In ancient times, and also still today in some...",175157,4,1,39
175158,4C471936CD75,7,13,0,903,905,911,913.0,"Of course,","In ancient times, and also still today in some...",175158,4,1,39


In [27]:
d[d.id=='00066EA9880D']

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
100290,00066EA9880D,1,0,5,0,77,0,77.0,Driverless cars are exaclty what you would exp...,Driverless cars are exaclty what you would exp...,100290,4,5,9
100291,00066EA9880D,1,1,3,77,98,77,100.0,The developement of these cars should be stopp...,Driverless cars are exaclty what you would exp...,100291,4,5,9
100292,00066EA9880D,1,2,0,98,106,100,108.0,One thing that the article mentions is that,Driverless cars are exaclty what you would exp...,100292,4,5,9
100293,00066EA9880D,1,3,1,106,124,108,126.0,the driver will be alerted when they will need...,Driverless cars are exaclty what you would exp...,100293,4,5,9
100294,00066EA9880D,1,4,2,124,243,126,247.0,This is such a dangerous thing because we all ...,Driverless cars are exaclty what you would exp...,100294,4,5,9
100295,00066EA9880D,1,5,1,243,258,247,262.0,Another thing that can go wrong with these car...,Driverless cars are exaclty what you would exp...,100295,4,5,9
100296,00066EA9880D,1,6,2,258,394,262,400.0,Every person with any kind of technological de...,Driverless cars are exaclty what you would exp...,100296,4,5,9
100297,00066EA9880D,1,7,0,394,410,400,416.0,One last thing that the article mentions is ne...,Driverless cars are exaclty what you would exp...,100297,4,5,9
100298,00066EA9880D,1,8,1,410,438,416,444.0,who to blame for the wreck if there were possi...,Driverless cars are exaclty what you would exp...,100298,4,5,9
100299,00066EA9880D,1,9,2,438,560,444,568.0,Should the manufacturer of the car be blamed o...,Driverless cars are exaclty what you would exp...,100299,4,5,9


In [28]:
d.columns

Index(['id', 'part', 'para_id', 'para_type', 'start', 'end', 'start2', 'end2',
       'para', 'text', 'pid', 'kfold', 'cluster', 'worker'],
      dtype='object')

In [29]:
d.to_csv('../input/feedback-prize-2021/train_en.csv')

In [30]:
d.reset_index().to_feather('../input/feedback-prize-2021/train_en.fea')

In [31]:
pd.read_feather('../input/feedback-prize-2021/train_en.fea')

,index,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
0,0,423A1CA112E2,13,0,0,0,1,0,3.0,Phones [BR] [BR],Phones [BR] [BR] Modern humans today are alway...,0,2,8,67
1,1,423A1CA112E2,13,1,5,1,45,3,47.0,Modern humans today are always on their phone....,Phones [BR] [BR] Modern humans today are alway...,1,2,8,67
2,2,423A1CA112E2,13,2,3,45,60,47,62.0,They are some really bad consequences when stu...,Phones [BR] [BR] Modern humans today are alway...,2,2,8,67
3,3,423A1CA112E2,13,3,2,60,76,62,80.0,Some certain areas in the United States ban ph...,Phones [BR] [BR] Modern humans today are alway...,3,2,8,67
4,4,423A1CA112E2,13,4,2,76,139,80,145.0,"When people have phones, they know about certa...",Phones [BR] [BR] Modern humans today are alway...,4,2,8,67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175155,175155,4C471936CD75,7,10,2,576,828,582,834.0,seeking multiple opinions before making a hard...,"In ancient times, and also still today in some...",175155,4,1,39
175156,175156,4C471936CD75,7,11,3,828,839,834,845.0,it is better to seek multiple opinions instead...,"In ancient times, and also still today in some...",175156,4,1,39
175157,175157,4C471936CD75,7,12,2,839,903,845,911.0,The impact of asking people to help you make a...,"In ancient times, and also still today in some...",175157,4,1,39
175158,175158,4C471936CD75,7,13,0,903,905,911,913.0,"Of course,","In ancient times, and also still today in some...",175158,4,1,39


In [32]:
d.describe()

,part,para_id,para_type,start,end,start2,end2,pid,kfold,cluster,worker
count,175160.000000,175160.000000,175160.000000,175160.000000,175160.000000,175160.000000,159566.000000,175160.000000,175160.000000,175160.000000,175160.000000
mean,9.023419,5.826250,2.019120,169.021169,206.524583,172.979185,189.884023,87579.500000,1.997985,6.530858,47.115078
std,5.482194,4.274395,1.736715,162.531740,177.146203,165.504884,163.886686,50564.480913,1.417359,4.293917,27.453096
min,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,4.000000,2.000000,1.000000,48.000000,69.000000,49.000000,65.000000,43789.750000,1.000000,3.000000,24.000000
50%,9.000000,5.000000,2.000000,119.000000,159.000000,122.000000,142.000000,87579.500000,2.000000,6.000000,47.000000
75%,14.000000,9.000000,3.000000,250.000000,297.000000,257.000000,273.000000,131369.250000,3.000000,10.000000,71.000000
max,18.000000,31.000000,7.000000,1369.000000,1656.000000,1465.000000,1465.000000,175159.000000,4.000000,14.000000,94.000000


In [33]:
d.kfold.value_counts()

1    35773
4    35469
0    34958
2    34562
3    34398
Name: kfold, dtype: int64

In [34]:
d.part.value_counts()

15    9810
17    9756
8     9554
1     9495
10    9468
11    9374
0     9374
7     9312
5     9277
16    9271
13    9199
14    9156
2     9151
6     9115
12    8900
3     8827
9     8790
18    8674
4     8657
Name: part, dtype: int64

In [35]:
d.worker.value_counts()

40    2233
59    2145
71    2108
85    2097
3     2079
      ... 
30    1573
19    1564
48    1539
18    1531
21    1285
Name: worker, Length: 95, dtype: int64

In [36]:
len(d.worker == 0)

175160

In [37]:
id = 'C196DA0C7688'
d[d.id==id]

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
54161,C196DA0C7688,17,0,5,0,24,0,26.0,The author has been studying Venus to find out...,The author has been studying Venus to find out...,54161,2,2,87
54162,C196DA0C7688,17,1,2,24,185,26,191.0,The author says that Venus has many feature li...,The author has been studying Venus to find out...,54162,2,2,87
54163,C196DA0C7688,17,2,3,185,222,191,NaN,"Therefore, Venus may be wirth to study it but ...",The author has been studying Venus to find out...,54163,2,2,87


In [38]:
d[d.id=='BE01ACCDF251']

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
14088,BE01ACCDF251,12,0,0,0,7,0,7.0,According to the Articles fellow citizens about,According to the Articles fellow citizens abou...,14088,3,11,63
14089,BE01ACCDF251,12,1,3,7,18,7,18.0,the advantages of limiting car usage are going...,According to the Articles fellow citizens abou...,14089,3,11,63
14090,BE01ACCDF251,12,2,0,18,19,18,19.0,because,According to the Articles fellow citizens abou...,14090,3,11,63
14091,BE01ACCDF251,12,3,2,19,122,19,124.0,the article indacties 70 percemt of Vauban's f...,According to the Articles fellow citizens abou...,14091,3,11,63
14092,BE01ACCDF251,12,4,0,122,124,124,126.0,"One Reason,",According to the Articles fellow citizens abou...,14092,3,11,63
14093,BE01ACCDF251,12,5,2,124,241,126,245.0,Fellow citizens about the advantages of limiti...,According to the Articles fellow citizens abou...,14093,3,11,63
14094,BE01ACCDF251,12,6,0,241,243,245,247.0,"Another Reason,fellow",According to the Articles fellow citizens abou...,14094,3,11,63
14095,BE01ACCDF251,12,7,2,243,325,247,331.0,citizens about the advantages of limiting car ...,According to the Articles fellow citizens abou...,14095,3,11,63
14096,BE01ACCDF251,12,8,2,325,396,331,404.0,citizens about the advantages of limiting car ...,According to the Articles fellow citizens abou...,14096,3,11,63
14097,BE01ACCDF251,12,9,4,396,436,404,NaN,"Finally, these are the reason why fellow citiz...",According to the Articles fellow citizens abou...,14097,3,11,63


In [39]:
d[d.id==id].text.values[0]

'The author has been studying Venus to find out if humans can live there and there are some reason that humans cant live there. [BR] [BR] The author says that Venus has many feature like earth and that it may be possible for humans to live there. Even though they have seen what could happen like the bad part of living there, they are still trying to find a way to make humans live there. Venus is like Earths "twin" because it is the closest planet to the earth unlike the other planets. Also, Venus atmosphere of carbon dioxide is 97 percent. [BR] [BR] Venus may be worth studying for but despite the danger that it may have to any human could be bad and even though the author says that they are going to have to make humans a part of there studyies so they can find a way to live there. In the text it says that "numerous factors contribute to Venus\'s reputation as a challenging planet for humans to study, despite the proximity to us" saying that they will have to study humans no matter what

In [40]:
gkeys = ['id', 'text', 'kfold', 'cluster', 'worker', 'part']
allkeys = gkeys + ['para_type', 'start', 'end']
dflat = d[allkeys].groupby(gkeys).agg(list).reset_index()
dflat

,id,text,kfold,cluster,worker,part,para_type,start,end
0,0000D23A521A,"Some people belive that the so called ""face"" o...",0,7,10,2,"[3, 2, 2, 1, 6, 7, 2, 4]","[0, 34, 69, 84, 117, 134, 154, 186]","[34, 69, 84, 117, 134, 154, 186, 251]"
1,00066EA9880D,Driverless cars are exaclty what you would exp...,4,5,9,1,"[5, 3, 0, 1, 2, 1, 2, 0, 1, 2, 4]","[0, 77, 98, 106, 124, 243, 258, 394, 410, 438,...","[77, 98, 106, 124, 243, 258, 394, 410, 438, 56..."
2,000E6DE9E817,Dear: Principal [BR] [BR] I am arguing against...,3,13,3,0,"[0, 3, 0, 6, 7, 2, 0, 1, 2, 1, 0, 1, 0, 1, 0, ...","[0, 2, 9, 10, 26, 54, 73, 76, 96, 137, 172, 17...","[2, 9, 10, 26, 54, 73, 76, 96, 137, 172, 177, ..."
3,001552828BD0,Would you be able to give your car up? Having ...,3,11,78,15,"[5, 2, 1, 1, 2, 1, 2, 1, 2, 3]","[0, 32, 168, 184, 228, 289, 302, 366, 384, 484]","[32, 168, 184, 228, 289, 302, 366, 384, 484, 512]"
4,0016926B079C,I think that students would benefit from learn...,3,9,43,8,"[3, 1, 1, 0, 1, 1, 2, 1, 2, 1, 1, 2]","[0, 10, 16, 28, 37, 42, 48, 104, 125, 170, 202...","[10, 16, 28, 37, 42, 48, 104, 125, 170, 202, 2..."
...,...,...,...,...,...,...,...,...,...
15589,FFF1442D6698,"Every student looks forward to summer break, i...",0,12,60,12,"[5, 3, 1, 1, 0, 2, 0, 2, 6, 7, 0, 2, 0, 4]","[0, 111, 123, 134, 141, 142, 207, 219, 355, 37...","[111, 123, 134, 141, 142, 207, 219, 355, 371, ..."
15590,FFF1ED4F8544,Many citizens argue that the Electoral college...,2,4,77,15,"[5, 3, 1, 1, 1, 1, 2, 1, 2]","[0, 60, 78, 86, 93, 102, 128, 293, 298]","[60, 78, 86, 93, 102, 128, 293, 298, 412]"
15591,FFF868E06176,"Every summer break, students are given project...",2,12,62,12,"[5, 3, 0, 1, 2, 0, 1, 2, 1, 2, 4]","[0, 39, 56, 71, 87, 165, 177, 191, 244, 270, 383]","[39, 56, 71, 87, 165, 177, 191, 244, 270, 383,..."
15592,FFFD0AF13501,"In the article ""A Cowboy Who Rode the Waves"" L...",4,6,24,4,"[0, 1, 1, 2, 1, 1, 2, 0, 3, 4]","[0, 44, 52, 66, 83, 102, 116, 191, 193, 209]","[44, 52, 66, 83, 102, 116, 191, 193, 209, 250]"


In [41]:
dflat.reset_index().to_feather('../input/feedback-prize-2021/train_flat_en.fea')

In [42]:
dflat[dflat.id=='00066EA9880D']

,id,text,kfold,cluster,worker,part,para_type,start,end
1,00066EA9880D,Driverless cars are exaclty what you would exp...,4,5,9,1,"[5, 3, 0, 1, 2, 1, 2, 0, 1, 2, 4]","[0, 77, 98, 106, 124, 243, 258, 394, 410, 438,...","[77, 98, 106, 124, 243, 258, 394, 410, 438, 56..."


In [43]:
d[d.id=='0016926B079C']

,id,part,para_id,para_type,start,end,start2,end2,para,text,pid,kfold,cluster,worker
149065,0016926B079C,8,0,3,0,10,0,10.0,I think that students would benefit from learn...,I think that students would benefit from learn...,149065,3,9,43
149066,0016926B079C,8,1,1,10,16,10,16.0,they wont have to change and,I think that students would benefit from learn...,149066,3,9,43
149067,0016926B079C,8,2,1,16,28,16,28.0,get up early in the morning to shower and do t...,I think that students would benefit from learn...,149067,3,9,43
149068,0016926B079C,8,3,0,28,37,28,37.0,taking only classes helps them because at ther...,I think that students would benefit from learn...,149068,3,9,43
149069,0016926B079C,8,4,1,37,42,37,42.0,they'll be pay more attention.,I think that students would benefit from learn...,149069,3,9,43
149070,0016926B079C,8,5,1,42,48,42,50.0,they will be comfortable at home. [BR] [BR],I think that students would benefit from learn...,149070,3,9,43
149071,0016926B079C,8,6,2,48,104,50,106.0,The hardest part of school is getting ready. y...,I think that students would benefit from learn...,149071,3,9,43
149072,0016926B079C,8,7,1,104,125,106,129.0,with the online classes you can wear anything ...,I think that students would benefit from learn...,149072,3,9,43
149073,0016926B079C,8,8,2,125,170,129,174.0,most students usually take showers before scho...,I think that students would benefit from learn...,149073,3,9,43
149074,0016926B079C,8,9,1,170,202,174,208.0,when u have online classes u wont need to miss...,I think that students would benefit from learn...,149074,3,9,43
